# Private Set Intersection using Secure Multiparty Computation

This example demonstrates using Secure Multiparty Computation (MPC) to compare two secret lists and show which items occur in both lists. This makes it possible, for instance, for two researchers to compare lists of people and to see who occurs on both lists. The useful part is that this process is carried out in a way that keeps the non-matching names secret from the other party.

As a concrete example, consider two researchers, Alice and Bob. Alice has a list of three people: Charlie, David, and Edna. Bob has a list of four people: Fred, David, Gretta, and Herbie. Using Private Set Intersection (PSI), Alice and Bob can learn that David is on both lists. Importantly, this is done without Bob learning about Charlie or Edna, and without Alice learning about Fred, Gretta, or Herbie.



## A Worked Example

To begin, we'll need to run two of these notebooks at the same time - one for each party. One instance will be "Party Zero", and the other one will be "Party One". Party Zero is the one who will actually see the results of the PSI.



### Step 0: Configuration details

Edit the following variables to describe your environment (and that of your collaborator). We'll need to know, for instance, if we are Party 0 (the one that sees the final results) or Party 1 (the other person). We'll also need to know everyone's IP address.

In [ ]:
myPartyNumber = 0

myIPaddress = "18.191.213.3"
otherIPaddress = "18.222.23.230"
spdzPort = "5000"


In [ ]:
%%bash -s $myIPaddress $otherIPaddress
echo $1 > $HOME/parties
echo $2 >> $HOME/parties

### Step 1: Read the names and hash them
PSI works with numbers instead of with text. This isn't a problem - we just need to convert our lines of text into numbers in a process called "hashing". In the following code block, we'll open a file, read the strings from it, and use a hashing function called "SHA256". Running this hashing function can turn strings into some really big numbers - up to eighteen billion billion. We'll shrink those numbers down to a maximum of two billion before we use them for anything.

In [ ]:
import hashlib

def makeShortHash (input):
    hasher = hashlib.sha256(input.encode('utf-8'))
    hashedResults = hasher.digest()
    # convert the 32 bytes we got back into a 4 byte (32 bit) integer
    # and strip the MSB to force it to be positive.
    # odds of a hash collision about 1:2.15 billion.
    shorterNum = (  ((hashedResults[0] & 127) << 24) |
             (hashedResults[1] << 16) |
             (hashedResults[2] << 8) |
             (hashedResults[3] ) )
    return shorterNum

In [ ]:
filenameToProcess = "/home/ec2-user/privateSet.txt"
myPrivateSet = []
f = open(filenameToProcess, "r")
for rawInputLine in f:
  inputLine = rawInputLine.strip()
  if len(inputLine) > 0 :
    myPrivateSet.append((inputLine, makeShortHash(inputLine)))
    
for iterTuple in myPrivateSet:
    print (iterTuple)
    

### Step 2: Count the total number of names for this party

The number of names in each party's list doesn't have to be the same (and rarely will be). Count them.


In [ ]:
totalSecretEntries = len(myPrivateSet)
print ("There are " + str(totalSecretEntries) + " names that will be submitted by this party.")

### Step 3: Gather information about the other party

We need to do two things:
* Gather the IP address of the other party.
* Put the IP address of Party 0, followed by the IP address of Party 1 on the next line, into a file in our home directories named "parties".
* Find out how many items our colleague has in their list and save it in the variable "otherPartyNumberOfLines".

No fancy technology for this one. Maybe call them on the phone?


In [ ]:
otherPartyNumberOfLines = 4

### Step 4: (Automatically!) create a MPC program to do the comparision

Now that we know how many entries each party wants to compare, we have enough information to automatically write and compile a MPC function to do the comparision. We need to write the function on the fly because of how allocating memory works in the SPDZ/2 implementation. Then we will need to compile the program so we can actually run it.

In [ ]:
#import tempfile
#f = tempfile.NamedTemporaryFile(delete=False)
f = open("/home/ec2-user/SPDZ-2/Programs/Source/psi.mpc","w")


preamble = """# (C) 2018 University of Bristol. See License.txt
# Modified by renci.org for practical use but still
# rather heavily influenced by the original
# Modified starting from:
#Example programs used in the SPDZ tutorial at the TPMPC 2017 workshop in Bristol.

from util import if_else
program.bit_length = 32


def compute_intersection(a, b):
        #Naive (as in "not an index join, but OK") quadratic private set intersection.

        #Returns: secret Array with intersection (padded to len(a)), and
        #secret Array of bits indicating whether Party Zero's input matches or not

        # get the lengths of input arrays, make the "left" array which ever one is actually
        # shorter, and set aLen and bLen to be the lengths of the shorter and longer 
        # arrays, respectively.
        aLen = len(a)
        bLen = len(b)

        intersection = Array(aLen, sint)
        is_match_at = Array(aLen, sint)

        @for_range(aLen)
        def _(i):
                @for_range(bLen)
                def _(j):
                        match = a[i] == b[j]
                        is_match_at[i] += match
                        intersection[i] = if_else(match, a[i], intersection[i])
        return intersection, is_match_at


def set_intersection(n,p):
        a = Array(n, sint)
        b = Array(p, sint)

"""
    
f.write(preamble)



#for i in range (totalSecretEntries):
#    f.write("        a[" + str(i) + "] = sint.get_input_from(0)\n")
#for i in range (otherPartyNumberOfLines):
#    f.write("        b[" + str(i) + "] = sint.get_input_from(1)\n")


# we need to put the first party (party0's) items first, then the second one.
# find out if we're 0 or 1, then generate the input section

partylen=[]
if myPartyNumber == 0:
    partylen.append(totalSecretEntries)
    partylen.append(otherPartyNumberOfLines)
else:
    partylen.append(otherPartyNumberOfLines)
    partylen.append(totalSecretEntries)

print("How many party entries:" + str(partylen[0]) + " " + str(partylen[1]))


for i in range (partylen[0]):
    f.write("        a[" + str(i) + "] = sint.get_input_from(0)\n")
for i in range (partylen[1]):
    f.write("        b[" + str(i) + "] = sint.get_input_from(1)\n")

postamble = """        intersection, is_match_at = compute_intersection(a,b)

        print_ln('Printing set intersection (0: not in intersection)')
        size = MemValue(sint(0))
        total = MemValue(sint(0))

        resultLength = min(n,p)
        @for_range(resultLength)
        def _(i):
                size.write(size + is_match_at[i])
                total.write(total + intersection[i])
                print_str('%s\\n', intersection[i].reveal())

"""
f.write(postamble)
#f.write("set_intersection(" + str (totalSecretEntries) + "," + str(otherPartyNumberOfLines) +")\n")
f.write("set_intersection(" + str (partylen[0]) + "," + str(partylen[1]) +")\n")


f.close()

### Step 5: Compile the above-generated program
The program, with the requisite number of statements to read valuies from each party, has been saved as "psi.mpc" in the correct SPDZ directory. Now, compile it to bytecodes.

In [ ]:
%%bash
# change to the SPDZ directory
cd $HOME/SPDZ-2
# compile the function. DO NOT INCLUDE THE ".mpc" FILE EXTENSION
#./compile.py tripleadd
./compile.py psi

### Step 6: Run the secure multiparty computation
All the preparation has been done. It's time to run the program now and give it some input. First we will iterate through the hashed values we calculated and save them in a temporary file. Then we will run the MPC calculation and capture the output (and any error messages) into the string variables mpcOut and mpcErr.

In [ ]:
import tempfile
tf = tempfile.NamedTemporaryFile(delete=False)   # don't automatically delete on close.
for iterTuple in myPrivateSet:
    print(iterTuple)
    print(iterTuple[1])
    tf.write((str(iterTuple[1]) + "\n").encode('utf-8'))
tempFileName = tf.name
tf.close()  # note: explicitly not deleted yet!

In [ ]:
%%bash -s $tempFileName $myPartyNumber --out mpcOut --err mpcErr
cd $HOME/SPDZ-2
cat $1 | $HOME/SPDZ-2/Player-Online.x -ip $HOME/parties $2 psi


### Step 7: Profit! 
The computation has run - with any luck, it ran sucessfully. Delete the input file of hashes, then check the output and error strings to see if anything went haywire.

In [ ]:
%%bash -s $tempFileName
rm $1

In [ ]:

print ("Output from the secure multiparty computation: " + mpcOut)
print("SEPARATOR   XXXXXXX    SEPARATOR   XXXXXXX    SEPARATOR   XXXXXXX    ")
print("Error and debugging messages (even when successful, there will still be some debugging info). "+ mpcErr)


### Step 8: Look up the hashed value and see which names matched
What just came back was a list of hashed values and some zeros. The zeros mean "you had something here that didn't match". In our example above, we have a 0, then 649415712, and then finally another zero. This means "the first one didn't match, the second one did (and its hashed value is 649415712), and the third one didn't". It's up to you to decide how to handle this - you could keep track of position numbers (in our example, the second line isn't a zero, so the second name in our list is the matching one) or we could look through the myPrivateSet variable and find the tuple that matches and take it apart. Either way is perfectly acceptable. Since this is a tutorial example, let's do it the easy way and match things up by position number.

In [ ]:
if myPartyNumber == 0:      # only Party 0 gets the output in this example.
    matchedHashes = mpcOut.splitlines()
    firstLine = True
    matchedArrayIndex = 0
    for matched in matchedHashes:
        if firstLine:
            firstLine = False
        else:
            if matched != "0":
                print((matched, matchedArrayIndex, myPrivateSet[matchedArrayIndex][0]))
            matchedArrayIndex = matchedArrayIndex + 1